## Navigating options market data with the OpenBB Platform

In [1]:
from IPython.display import display
from openbb import obb

In [2]:
obb.user.preferences.output_type = "dataframe"

Fetches options chains for the symbol "SPY" and stores it in 'chains'

In [3]:
chains = obb.derivatives.options.chains(symbol="SPY")

In [4]:
display(chains.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9910 entries, 0 to 9909
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   underlying_symbol   9910 non-null   object        
 1   underlying_price    9910 non-null   float64       
 2   contract_symbol     9910 non-null   object        
 3   expiration          9910 non-null   object        
 4   dte                 9910 non-null   int64         
 5   strike              9910 non-null   float64       
 6   option_type         9910 non-null   object        
 7   open_interest       9910 non-null   int64         
 8   volume              9910 non-null   int64         
 9   theoretical_price   9910 non-null   float64       
 10  last_trade_price    9910 non-null   float64       
 11  last_trade_time     8197 non-null   datetime64[ns]
 12  tick                9910 non-null   object        
 13  bid                 9910 non-null   float64     

None

Fetches historical price data for the specified option symbol using the "yfinance" provider and selects the "close" and "volume" columns

In [5]:
historic = obb.equity.price.historical(
    symbol="SPY251219C00650000", provider="yfinance"
)[["close", "volume"]].head(10)

In [6]:
display(historic)

,close,volume
date,,
2024-02-22,6.30,29
2024-02-23,6.50,17
2024-02-26,6.39,12
2024-02-28,5.84,12
2024-02-29,6.05,0
2024-03-01,7.00,16
2024-03-04,7.33,479
2024-03-05,6.25,0
2024-03-07,7.80,3


## HDF5. Save and load data. Vanilla

In [7]:
import pandas as pd

In [8]:
# Save data
h5 = pd.HDFStore("data.h5", "w")
h5["data"] = historic
h5.close()
!ls -n data.*

-rw-r--r-- 1 0 0 8000 Feb 21 14:30 data.h5


In [9]:
# Load data
h5 = pd.HDFStore("data.h5", "r")
data_copy = h5["data"]
h5.close()
data_copy.head()
!rm data.h5 # Remove the file

## HDF5. Save and load data. Compression

In [10]:
## Storing in HDF5 with compression
h5 = pd.HDFStore("data.h5", "w", complevel=9, complib="blosc")
h5["data"] = historic
h5.close()
!ls -n data.*

-rw-r--r-- 1 0 0 22115 Feb 21 14:30 data.h5


In [11]:
## Reading from HDF5 with compression
h5 = pd.HDFStore("data.h5", "r", complevel=9, complib="blosc")
data_copy2 = h5["data"]
h5.close()
!rm data.h5 # Remove the file
data_copy2.head()

,close,volume
date,,
2024-02-22,6.30,29
2024-02-23,6.50,17
2024-02-26,6.39,12
2024-02-28,5.84,12
2024-02-29,6.05,0


## HDF5. Save and load data. Compression and chunking

In [12]:
## Storing in HDF5 with compression
h5 = pd.HDFStore("data.h5", "w", complevel=9, complib="blosc", chunksize=1000)
h5["data"] = historic
h5.close()
!ls -n data.*

-rw-r--r-- 1 0 0 22115 Feb 21 14:31 data.h5


In [13]:
## Reading from HDF5 with compression
h5 = pd.HDFStore("data.h5", "r", complevel=9, complib="blosc", chunksize=1000)
data_copy3 = h5["data"]
h5.close()
!rm data.h5 # Remove the file
data_copy3.head()

,close,volume
date,,
2024-02-22,6.30,29
2024-02-23,6.50,17
2024-02-26,6.39,12
2024-02-28,5.84,12
2024-02-29,6.05,0


## SQLite3. Save and load data

In [14]:
import sqlite3 as sq3

In [15]:
## Storing in SQLite3
conn = sq3.connect("data.db")  # Create a connection to the database
historic.to_sql(
    "data", conn, if_exists="replace", index=False
)  # Store the data in the database
conn.close()  # Close the connection
!ls -n data.*

-rw-r--r-- 1 0 0 8192 Feb 21 14:31 data.db


In [16]:
## Reading from SQLite3
conn = sq3.connect("data.db")  # Create a connection to the database
data_copy4 = pd.read_sql_query(
    "SELECT * FROM data", conn
)  # Read the data from the database
conn.close()  # Close the connection
!rm data.db # Remove the file
data_copy4.head()

,close,volume
0,6.30,29
1,6.50,17
2,6.39,12
3,5.84,12
4,6.05,0
